In [3]:
from itertools import islice
import six
from pyspark import SparkContext, SparkConf




conf = SparkConf().setAppName("Simple Masking App").setMaster("local")
sc = SparkContext(conf=conf)

# Option 1: Маска для email

In [4]:
#функция маскировки
def maskEmail(email):
    firstPart = list(email.split('@')[0])
    secondPart = list(email.split('@')[1])
    f = ['*' for sub in firstPart[1:]] 
    s = ['*' for sub in secondPart[1:"".join(secondPart).find('.')]] 
    str1="".join(firstPart[0:1])+"".join(f)
    str2="".join(secondPart[0:1])+"".join(s)+"".join(secondPart["".join(secondPart).find('.'):])
    return str1+'@'+str2

#Test
str = 'james@gmail.com'
maskEmail(str)

'j****@g****.com'

In [11]:
empRDD = sc.textFile("EmployData.csv")

#Print RDD data
for i in empRDD.take(10):print(i)

James,james@gmail.com,Sr. Developer
Smith,Smith@gmail.com,Project Lead
Vasisualii,VS@mail.com,Data engineer


In [12]:
#Применяем
empRDD1 = empRDD.map(lambda x:(x.split(",")[0],maskEmail(x.split(",")[1]),x.split(",")[2]))
for i in empRDD1.take(2):print(i)

('James', 'j****@g****.com', 'Sr. Developer')
('Smith', 'S****@g****.com', 'Project Lead')


# Option 2: Маскировка колонок

In [13]:
#функция макскировки
def maskColumn(colName):
    if isinstance(colName, six.string_types) == True:
        return '*****Masked Data*****'
    else:
        return 0

In [14]:
maskColumn("Confidential Data")

'*****Masked Data*****'

In [15]:
maskColumn(35440)

0

In [16]:
maskColumn(35440.766)

0

In [17]:
empRDD1 = empRDD.map(lambda x:(x.split(",")[0],maskColumn(x.split(",")[1]),maskColumn(x.split(",")[2])))
for i in empRDD1.take(2):print(i)

('James', '*****Masked Data*****', '*****Masked Data*****')
('Smith', '*****Masked Data*****', '*****Masked Data*****')


# Option 3: Маскировка и размаскировка

In [23]:
import functools
def toHex(s):
    lst = []
    for ch in s:
        hv = hex(ord(ch)).replace('0x', '')
        if len(hv) == 1:
            hv = '0'+hv
        lst.append(hv)
        
    return functools.reduce(lambda x,y:x+y, lst)

#hex размаскировка
def toStr(s):
    return s and chr(int(s[:2], base=16)) + toStr(s[2:]) or ''

In [21]:
toHex('Test Data')

'546573742044617461'

In [22]:
toStr('546573742044617461')

'Test Data'